In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import target_encoding as te
import gc

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 
import time

#Images
from PIL import Image
import zipfile

In [2]:
train_df = pd.read_csv("/home/jonas/Documents/Uni/DataChallenge/train.csv", parse_dates=["activation_date"], nrows=1000)
test_df = pd.read_csv("/home/jonas/Documents/Uni/DataChallenge/test.csv", parse_dates=["activation_date"], nrows=1000)
trainindex = train_df.index
testindex = test_df.index
test_id = test_df["item_id"].values
print("Train file rows and columns are : ", train_df.shape)
print("Test file rows and columns are : ", test_df.shape)


Train file rows and columns are :  (1000, 18)
Test file rows and columns are :  (1000, 17)


In [3]:
train_y = train_df.deal_probability.copy()
train_df.drop("deal_probability",axis=1, inplace=True)
# Target encode the categorical variables #
cat_vars = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3", "image_top_1"]
for col in cat_vars:
    train_df[col], test_df[col] = te.target_encode(train_df[col], test_df[col], train_y, min_samples_leaf=100, smoothing=10, noise_level=0.01)

In [4]:
# Image Featues
cols_to_add = ['width', 'height', 'img_mean_color', 'img_std_color']
pd.concat([train_df,pd.DataFrame(columns=cols_to_add)])
pd.concat([test_df,pd.DataFrame(columns=cols_to_add)])

,activation_date,category_name,city,description,height,image,image_green_std,image_red_std,image_top_1,img_blue_mean,...,param_1,param_2,param_3,parent_category_name,price,region,title,user_id,user_type,width
0,2017-03-28,0.148597,0.148687,"Кокон для сна малыша,пользовались меньше месяц...",NaN,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,NaN,NaN,0.147401,NaN,...,0.147707,0.146248,0.150479,0.067493,400.0,0.151124,Кокоби(кокон для сна),e00f8ff2eaf9,0.162208,NaN
1,2017-03-26,0.150116,0.147326,"Стойка для одежды, под вешалки. С бутика.",NaN,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,NaN,NaN,0.146399,NaN,...,0.149339,0.145623,0.145819,0.191048,3000.0,0.146898,Стойка для Одежды,39aeb48f0017,0.161656,NaN
2,2017-03-20,0.145757,0.147602,"В хорошем состоянии, домашний кинотеатр с blu ...",NaN,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,NaN,NaN,0.146778,NaN,...,0.148806,0.148143,0.146454,0.156376,4000.0,0.147449,Philips bluray,91e2f88dd6e3,0.163983,NaN
3,2017-03-25,0.148206,0.149280,Продам кресло от0-25кг,NaN,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,NaN,NaN,0.148460,NaN,...,0.144859,0.148029,0.148266,0.067584,2200.0,0.150294,Автокресло,bf5cccea572d,0.113188,NaN
4,2017-03-16,0.148347,0.148972,Все вопросы по телефону.,NaN,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,NaN,NaN,0.147114,NaN,...,0.148123,0.146151,0.146006,0.149415,40000.0,0.148089,"ВАЗ 2110, 2003",ef50846afc0b,0.165683,NaN
5,2017-03-28,0.149272,0.148730,В хорошем состоянии,NaN,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,NaN,NaN,0.146823,NaN,...,0.151093,0.147773,0.146643,0.068799,1300.0,0.148973,Авто люлька,bbfad0b1ad0a,0.164915,NaN
6,2017-03-23,0.149186,0.147827,Электро водонагреватель накопительный на 100 л...,NaN,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,NaN,NaN,0.148722,NaN,...,0.147370,0.149697,0.147541,0.190841,11000.0,0.147307,Водонагреватель 100 литров нержавейка плоский,08f469d2e6f7,0.165284,NaN
7,2017-03-25,0.039982,0.147765,Бойфренды в хорошем состоянии.,NaN,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,NaN,NaN,0.148291,NaN,...,0.040075,0.147351,0.149213,0.066935,500.0,0.147935,Бойфренды colins,fef86baa002c,0.163403,NaN
8,2017-03-17,0.039623,0.147255,54 раз мер очень удобное,NaN,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,NaN,NaN,0.147689,NaN,...,0.040517,0.149058,0.145340,0.067261,500.0,0.145248,Платье,055825270190,0.165861,NaN
9,2017-03-22,0.051586,0.146891,По стельке 15.5см мерить приокский район. Цвет...,NaN,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,NaN,NaN,0.145120,NaN,...,0.125957,0.101574,0.147604,0.068219,400.0,0.151189,Полу ботиночки замш натур.Бамбини,f9e8f831d94c,0.114583,NaN


In [5]:
def get_image_features(df, image_id, image):
    dat = np.array(image)
    df.loc[df['image'] == image_id, 'width'] = dat[0].size[0]
    df.loc[df['image'] == image_id, 'height'] = dat[0].size[1]
    df.loc[df['image'] == image_id, 'img_mean_color'] = np.mean(dat[1].flatten())
    df.loc[df['image'] == image_id, 'img_std_color'] = np.std(dat[1].flatten())
    return    

In [ ]:
train_zip = zipfile.ZipFile('../input/avito-demand-prediction/train_jpg.zip', 'r')
test_zip = zipfile.ZipFile('../input/avito-demand-prediction/test_jpg.zip', 'r')
zips = [train_zip, test_zip]
def insert_image_features(zipfile)
    df = test_df
    if(zipfile == train_zip):
        df = train_df
    files_in_zip = sorted(zipfile.namelist())
    for i,filename in enumerate(files_in_zip):
        if filename.endswith('.jpg'):
            file = train_zip.open(files_in_zip[i])
            img = Image.open(file)
            image_id = file.split('/')[-1].split('.')[0]
            get_image_features(df, image_id, img)
return

executor = concurrent.futures.ProcessPoolExecutor(2)
futures = [executor.submit(try_my_operation, item) for task in zips]
concurrent.futures.wait(futures)
        
        

In [ ]:
# Combine train and test for rest of preprocessing

df = pd.concat([train_df,test_df],axis=0)
del train_df, test_df
gc.collect()

df.head()

In [ ]:
# Simple Feature Engineering

# Time Data
df["activation_weekday"] = df["activation_date"].dt.weekday
df["activation_monthday"] = df["activation_date"].dt.day

# Price
## Replace Nan with mean in price
#categories = df.category_name.unique()
#region = df.region.unique()
#param1 = df.param_1.unique()
#
#
#df["price_new"] = df["price"].values
#
#for cat in categories:
#    for reg in region:
#        cur_df = df.loc[(df["category_name"] == cat)  & (df["region"] == reg)]["price_new"]
#        cur_df.fillna(np.nanmean(cur_df.values), inplace=True)
#
#
#df["price"] = pd.isna(df["price"])
df["price"] = np.log(df["price"]+0.001)
df["price"].fillna(-999,inplace=True)
df["image_top_1"].fillna(-999,inplace=True)

#Drop Cols
cols_to_drop = ["item_id", "user_id", "activation_date", "image"]
df.drop(cols_to_drop, axis=1,inplace=True)

In [ ]:
# Text Features
df['text_feat'] = df.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features


# Meta Text Features
textfeats = ["description","text_feat", "title"]
for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('nicapotato') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words

In [ ]:
print("\n[TF-IDF] Term Frequency Inverse Document Frequency Stage")
russian_stop = set(stopwords.words('russian'))

tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}
def get_col(col_name): return lambda x: x[col_name]
vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=16000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('text_feat',CountVectorizer(
            ngram_range=(1, 2),
            #max_features=7000,
            preprocessor=get_col('text_feat'))),
        ('title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            #max_features=7000,
            preprocessor=get_col('title')))
    ])
    
start_vect=time.time()
vectorizer.fit(df.loc[trainindex,:].to_dict('records'))
ready_df = vectorizer.transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

# Drop Text Cols
df.drop(textfeats, axis=1,inplace=True)

In [ ]:
train_X = hstack([csr_matrix(df.head(trainindex.shape[0]).values),ready_df[0:trainindex.shape[0]]]) # Sparse Matrix
test_X = hstack([csr_matrix(df.tail(testindex.shape[0]).values),ready_df[trainindex.shape[0]:]])
tfvocab = df.columns.tolist() + tfvocab
for shape in [train_X,test_X]:
    print("{} Rows and {} Cols".format(*shape.shape))
print("Feature Names Length: ",len(tfvocab))
del df
gc.collect();

In [ ]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y, feature_name=tfvocab)
    lgval = lgb.Dataset(val_X, label=val_y, feature_name=tfvocab)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=20, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [ ]:

# Splitting the data for model training#
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)


# Training the model #
pred_test, model, evals_result = run_lgb(X_train, y_train, X_val, y_val, test_X)

# Making a submission file #
pred_test[pred_test>1] = 1
pred_test[pred_test<0] = 0
sub_df = pd.DataFrame({"item_id":test_id})
sub_df["deal_probability"] = pred_test
sub_df.to_csv("baseline_lgb.csv", index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
lgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()